In [1]:
# useing this variable for saving models and summaries
exp_name = "DWN9008_opp"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import hickle 

import sys
sys.path.append('../src')

In [3]:
import models.wavelet_conv_model as wavelet_conv_model
import training
import evaluation.nn_eval_activity as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '../data/opp6.hkl'
with open(data_path, 'r') as fin:
    data = hickle.load(fin)

In [5]:
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = data['train_gt']
validation_gt = data['validation_gt']
test_gt = data['test_gt']

In [6]:
train_data.shape, train_gt.shape

((1358, 1024, 77), (1358, 1024, 18))

In [7]:
test_data.shape, test_gt.shape

((233, 1024, 77), (233, 1024, 18))

## Model

In [8]:
# Model parameters
n_variables = 77
learning_rate = 0.001
batch_size = 100

wavelet_levels = 5    #number of wavelet layer to have in the network
convolution_levels = 3    #number of convolution to have in the network
pooling_stride = [4, 4, 4]
weights_shape = [[9, n_variables, 32],
                 [5, 32, 32],
                 [5, 32, 32]]
bias_shape = [32, 32, 32]
wavelet_weights_shape = [[5, n_variables, n_variables]]    #high_pass and low_pass dimensions
activation = tf.nn.relu
n_classes = 18

##positive weight for weighted cross entropy
s = train_gt.shape
resolution = np.prod(pooling_stride)
pooled_gt = np.reshape(train_gt[:,:(s[0]*s[1]/resolution)*resolution,:], (s[0]*(s[1]/resolution), resolution, s[2])).max(axis=1)
pos_weight = (1.0 - pooled_gt.mean(axis = 0)) / pooled_gt.mean(axis=0)


In [9]:
'''train_data = train_data[:,:-(train_data.shape[1]%((2**(wavelet_levels-1))*resolution)),:]
validation_data = validation_data[:,:-(validation_data.shape[1]%((2**(wavelet_levels-1))*resolution)),:]
test_data = test_data[:,:-(test_data.shape[1]%((2**(wavelet_levels-1))*resolution)),:]

train_gt = train_gt[:,:-(train_gt.shape[1]%((2**(wavelet_levels-1))*resolution)),:]
validation_gt = validation_gt[:,:-(validation_gt.shape[1]%((2**(wavelet_levels-1))*resolution)),:]
test_gt = test_gt[:,:-(test_gt.shape[1]%((2**(wavelet_levels-1))*resolution)),:]'''

'train_data = train_data[:,:-(train_data.shape[1]%((2**(wavelet_levels-1))*resolution)),:]\nvalidation_data = validation_data[:,:-(validation_data.shape[1]%((2**(wavelet_levels-1))*resolution)),:]\ntest_data = test_data[:,:-(test_data.shape[1]%((2**(wavelet_levels-1))*resolution)),:]\n\ntrain_gt = train_gt[:,:-(train_gt.shape[1]%((2**(wavelet_levels-1))*resolution)),:]\nvalidation_gt = validation_gt[:,:-(validation_gt.shape[1]%((2**(wavelet_levels-1))*resolution)),:]\ntest_gt = test_gt[:,:-(test_gt.shape[1]%((2**(wavelet_levels-1))*resolution)),:]'

In [10]:
train_data.shape

(1358, 1024, 77)

In [11]:
pos_weight.shape

(18,)

In [12]:
pos_weight

array([  0.1575919 ,  33.59872611,  33.82051282,  36.46206897,
        35.45637584,  26.22807018,  26.78516624,  39.3866171 ,
        41.27237354,  45.82758621,  50.98086124,  47.71748879,
        50.24528302,  42.63052209,  43.89256198,  33.16352201,
        10.44783983,  35.95238095])

In [13]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [14]:
summary_path = '../summary/' + exp_name 
checkpoint_path = '../model/' + exp_name + '/checkpoint' 

In [15]:
model = wavelet_conv_model.Model(n_variables = n_variables,
                                 learning_rate = learning_rate,
                                 n_classes = n_classes)

In [17]:
model.build_neural_wavelet_layer(pos_weight = pos_weight, 
                                 learning_rate = learning_rate,
                                 wavelet_levels = wavelet_levels,    
                                 convolution_levels = convolution_levels,    
                                 pooling_stride = pooling_stride,     
                                 weights_shape = weights_shape,
                                 bias_shape = bias_shape,
                                 wavelet_weights_shape = wavelet_weights_shape,    
                                 activation = activation,
                                 trainable_wavelet=True,
                                 trainable_conv=True,
                                 trainable_last=True)

In [18]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = test_data,
                    ground_truth_validation = test_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0,
                    eval_type = 'activity')

In [17]:
trainer.run_debug_ops()

{'wavelet_out_shape': [array([     1, 699392,     77], dtype=int32), array([     1, 349696,     77], dtype=int32), array([     1, 174848,     77], dtype=int32), array([    1, 87424,    77], dtype=int32), array([    1, 43712,    77], dtype=int32), array([    1, 21856,    77], dtype=int32)], 'cnn_out_shape': [array([    1, 10928,    18], dtype=int32), array([   1, 5464,   18], dtype=int32), array([   1, 2732,   18], dtype=int32), array([   1, 1366,   18], dtype=int32), array([  1, 683,  18], dtype=int32), array([  1, 342,  18], dtype=int32)], 'input_shape': array([     1, 699392,     77], dtype=int32), 'label_shape': array([     1, 699392,     18], dtype=int32)}


In [19]:
trainer.train(max_iter = 4000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Train Batch Evaluation
AUC = 0.088 vs. 0.079
F1 = 0.135 vs. 0.116
Accuracy = 0.017 vs. 0.056
Validation
AUC = 0.071 vs. 0.068
F1 = 0.105 vs. 0.098
Accuracy = 0.022 vs. 0.056
Minibatch Loss= 1.396609
Train Batch Evaluation
AUC = 0.545 vs. 0.080
F1 = 0.588 vs. 0.125
Accuracy = 0.500 vs. 0.056
Validation
AUC = 0.349 vs. 0.069
F1 = 0.415 vs. 0.095
Accuracy = 0.456 vs. 0.056
Minibatch Loss= 0.413774
Train Batch Evaluation
AUC = 0.684 vs. 0.081
F1 = 0.698 vs. 0.124
Accuracy = 0.691 vs. 0.056
Validation
AUC = 0.438 vs. 0.068
F1 = 0.483 vs. 0.093
Accuracy = 0.690 vs. 0.056
Minibatch Loss= 0.289055
Train Batch Evaluation
AUC = 0.766 vs. 0.082
F1 = 0.748 vs. 0.121
Accuracy = 0.814 vs. 0.056
Validation
AUC = 0.483 vs. 0.068
F1 = 0.512 vs. 0.097
Accuracy = 0.747 vs. 0.056
Minibatch Loss= 0.202670
Train Batch Evaluation
AUC = 0.810 vs. 0.080
F1 = 0.805 vs. 0.119
Accuracy = 0.839 vs. 0.056
Validation
AUC = 0.497 vs. 0.068
F1 = 0.523 vs. 0.091
Accuracy = 0.775 vs. 0.056
Minibatch Loss= 0.141721
Train

In [20]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

'../model/DWN9008_opp/checkpoint-4000'

In [ ]:
#load the model
saver = tf.train.Saver()
#saver.restore(sess, "../model" + "/CNN7002_Gleam" + "/CNN7002_Gleam")
saver.restore(sess, '../model/CNN7002_Gleam/CNN7002_Gleam-10000')


In [21]:
#This is the model evaluation on test dataset
_= nn_eval.evaluate_model (model, sess, test_data, test_gt)

AUC = 0.520 vs. 0.071
F1 = 0.575 vs. 0.109
Accuracy = 0.916 vs. 0.056


In [ ]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [ ]:
savemat('../poc/poc'+exp_name+'.mat', {'poc':probability_change})

In [21]:
1

1